In [60]:
import ccdproc
import os

from datetime import date
from astropy.units import second
from multiprocessing import cpu_count

def getMkdir(dirPath: str) -> str:
	if not os.path.exists(dirPath):
		os.mkdir(dirPath)
	return dirPath

MAX_PARALLEL = cpu_count() - 2

OBSV_DATE = date(year=2022, month=12, day=10).strftime("%Y-%m-%d")
BASE_DATA_DIR = "/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464"

BIAS_DIR =  os.path.join(BASE_DATA_DIR, "Bias", OBSV_DATE)
DARKS_DIR = os.path.join(BASE_DATA_DIR, "Darks", OBSV_DATE)
FLATS_DIR = os.path.join(BASE_DATA_DIR, "Flats", OBSV_DATE)

DATA_DIRS = [os.path.join(BASE_DATA_DIR, d) for d in os.listdir(BASE_DATA_DIR) if OBSV_DATE in d and os.path.isdir(os.path.join(BASE_DATA_DIR, d))]
CCD_KWARGS = {'unit': 'adu'}
DATA_DIRS

['/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10']

In [33]:
# Master Bias

biasImages = ccdproc.ImageFileCollection(BIAS_DIR)
masterBias = ccdproc.Combiner(biasImages.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()

biasOutDir = getMkdir(os.path.join(BIAS_DIR, "out"))
biasFilePath = os.path.join(biasOutDir, f"{OBSV_DATE}_MasterBias.fits")
masterBias.write(biasFilePath, overwrite=True)

In [34]:
# Master Dark

rawDarks = ccdproc.ImageFileCollection(DARKS_DIR)
biasCorrectedDir = getMkdir(os.path.join(DARKS_DIR, "bias_corrected"))

exposureTime: None
for dark, dark_fname in rawDarks.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	correctedDark = ccdproc.subtract_bias(dark, masterBias)
	exposureTime = correctedDark.header['EXPOSURE']
	correctedDark.write(os.path.join(biasCorrectedDir, f"b_{dark_fname}"), overwrite=True)

b_darks = ccdproc.ImageFileCollection(biasCorrectedDir)
masterDark = ccdproc.Combiner(b_darks.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()
masterDark.header['EXPOSURE'] = exposureTime

darkOutDir = getMkdir(os.path.join(DARKS_DIR, "out"))
masterDarkFilePath = os.path.join(darkOutDir, f"{OBSV_DATE}_MasterDark.fits")
masterDark.write(masterDarkFilePath, overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [35]:
# Master flat

rawFlats = ccdproc.ImageFileCollection(FLATS_DIR)
correctedFlatsDir = getMkdir(os.path.join(FLATS_DIR, "corrected"))

for flat, flat_fname in rawFlats.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	b_flat = ccdproc.subtract_bias(flat, masterBias)
	bd_flat = ccdproc.subtract_dark(b_flat, masterDark, exposure_time='EXPOSURE', exposure_unit=second, scale=True)
	bd_flat.write(os.path.join(correctedFlatsDir, f"bd_{flat_fname}"), overwrite=True)

bd_flats = ccdproc.ImageFileCollection(correctedFlatsDir)
masterFlat = ccdproc.Combiner(bd_flats.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()

flatOutDir = getMkdir(os.path.join(FLATS_DIR, "out"))
masterFlatFilePath = os.path.join(flatOutDir, f"{OBSV_DATE}_MasterFlat.fits")
masterFlat.write(masterFlatFilePath, overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [36]:
# Bias, Dark, Flat image correction

for dataDir in DATA_DIRS:
	correctedDir = getMkdir(os.path.join(dataDir, "corrected"))

	rawImages = ccdproc.ImageFileCollection(dataDir)
	for img, img_fname in rawImages.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
		b_img = ccdproc.subtract_bias(img, masterBias)
		bd_img = ccdproc.subtract_dark(b_img, masterDark, exposure_time='EXPOSURE', exposure_unit=second, scale=True)
		bdf_img = ccdproc.flat_correct(bd_img, masterFlat)
		bdf_img.write(os.path.join(correctedDir, f"bdf_{img_fname}"), overwrite=True)

In [59]:
# Plate solving

import subprocess
import datetime
import numpy as np

from multiprocessing import Pool, Lock

LOGGING_LOCK = Lock()
def printToLog(msg: str):
	with LOGGING_LOCK:
		with open(os.path.join(BASE_DATA_DIR, f"{OBSV_DATE}.log"), "a+") as logFile:
			logFile.write(f"[{datetime.datetime.now()}] {msg}\n")

SOLVE_CMD_TEMPLATE = "solve-field --fits-image --no-plots --timestamp --new-fits {solvedOutPath} --cpulimit 180 -D {tempFiles} {inputPath} --overwrite"
		
def plateSolveFile(correctedFile: str, correctedDir: str):
	print(correctedDir, correctedFile)
	plateSolveCmd = SOLVE_CMD_TEMPLATE.format(inputPath=os.path.join(correctedDir, correctedFile), 
											solvedOutPath=os.path.join(dataDir, "solved-fits", correctedFile),
											tempFiles=os.path.join(dataDir, "solved"))
	print(plateSolveCmd)
	result = subprocess.run(plateSolveCmd, shell=True)
	if result.returncode != 0:
		printToLog(f"Failed to plate solve {correctedFile}")

def plateSolve(correctedFiles: list[str], correctedDir: str):
	# print(f"{os.getpid()} {correctedFiles[0]} {len(correctedFiles)}\n")
	for cFile in correctedFiles:
		plateSolveFile(cFile, correctedDir)

class SolveCaller:
	def __init__(self, correctedDir: str) -> None:
		self.correctedDir = correctedDir

	def __call__(self, imagesChunks):
		plateSolve(imagesChunks, self.correctedDir)

for dataDir in DATA_DIRS:
	correctedDir = os.path.join(dataDir, "corrected") # output files from previous cell
	imagesChunks = np.array_split(os.listdir(correctedDir), MAX_PARALLEL)
	with Pool(MAX_PARALLEL) as pool:
		pool.map(SolveCaller(correctedDir), imagesChunks)

/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/corrected bdf_2022-12-10_18-44-29.fits
solve-field --fits-image --no-plots --timestamp --new-fits /run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/solved-fits/bdf_2022-12-10_18-44-29.fits --cpulimit 180 -D /run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/solved /run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/corrected/bdf_2022-12-10_18-44-29.fits --overwrite
[ 61471: 0.000] Reading input file 1 of 1: "/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/corrected/bdf_2022-12-10_18-44-29.fits"...
[ 61471: 0.001] Extracting sources...
/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/corrected bdf_2022-12-10_19-50-56.fits
solve-field --fits-image --no-plots --timestamp --new-fits /run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/solved-fits/bdf_2022-12-10_19-50-56.fits --c

In [15]:
# Shift plate solved images

from ccdproc import wcs_project
from astropy.wcs import WCS

from astropy.io import fits

targetWcs: WCS = None
for dataDir in DATA_DIRS:
	shiftedDir = getMkdir(os.path.join(dataDir, "shifted"))

	# rawImages = ccdproc.ImageFileCollection(dataDir)
	rawImages = ccdproc.ImageFileCollection("/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/solved-fits")
	for img, img_fname in rawImages.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
		if not targetWcs:
			targetWcs = WCS(img.header)
			# with fits.open(os.path.join(dataDir, img_fname)) as fitsImg:
			with fits.open(os.path.join("/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-12-10/solved-fits", img_fname)) as fitsImg:
				print(fitsImg)
				targetWcs = WCS(fitsImg.pop())
				print(targetWcs)
		else:
			print("Shifting image", img_fname)
			shiftedImg = wcs_project(img, targetWcs)
			shiftedImg.write(os.path.join(shiftedDir, f"s_{img_fname}"), overwrite=True)
			print(os.path.join(shiftedDir, f"s_{img_fname}"))

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 339.911684567  45.1766334944  
CRPIX : 1201.25222778  276.853923798  
CD1_1 CD1_2  : -9.79704510996e-05  4.41878965111e-06  
CD2_1 CD2_2  : -4.50572412468e-06  -9.80178245934e-05  
NAXIS : 1920  1200
Shifting image 2022-12-10_19-52-56.solved.fits


/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-10-27/shifted/s_2022-12-10_19-52-56.solved.fits
